# 🛳 Titanic: Machine Learning from Disaster
#### Цель: предсказать, кто выжил при крушении "Титаника" на основе информации о пассажирах.

Этот ноутбук является решением классической задачи классификации из соревнования на Kaggle. В процессе:

Загружаем и исследуем данные (train.csv и test.csv)
Обрабатываем пропущенные значения
Извлекаем важные признаки (титул, размер семьи, префикс каюты и др.)
Строим модель на основе GradientBoostingClassifier
Проводим настройку гиперпараметров через GridSearchCV
Делаем предсказания и сохраняем файл для отправки на Kaggle

In [1]:
import pandas as pd
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# Загрузка данных
df2 = pd.read_csv('test.csv')
df_train = pd.read_csv('train.csv')

# Функция предобработки данных
def preprocess_data(df):
    # Заполнение пропущенных значений
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    
    # Создание новых признаков
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    # Извлечение префикса из каюты
    letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
    df['Prefix'] = df['Cabin'].apply(lambda x: x[0] if (isinstance(x, str) and x[0] in letters) else 'No Value')
    
    # Извлечение титула из имени
    pattern = '(Mr\.|Mrs\.|Miss\.|Master\.|Dr\.|Rev\.)'
    df['Title'] = df['Name'].apply(lambda x: re.findall(pattern, x, flags=re.IGNORECASE))
    df['Title'] = df['Title'].apply(lambda x: x[0] if x else 'None')
    
    # Кодирование категориальных признаков
    df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Prefix', 'Title'], drop_first=True)
    
    return df

# Применение предобработки
df2 = preprocess_data(df2)
df_train = preprocess_data(df_train)

# Выбор признаков
features = ['Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone', 
            'Sex_male', 'Embarked_Q', 'Embarked_S', 'Prefix_B', 'Prefix_C', 
            'Prefix_D', 'Prefix_E', 'Prefix_F', 'Prefix_G', 'Title_Miss.', 
            'Title_Mr.', 'Title_Mrs.', 'Title_Rev.']

x_train = df_train[features]
y_train = df_train['Survived']
x_valid = df2[features]

# Разделение данных на обучающую и валидационную выборки
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Параметры для GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Модель для GridSearchCV
model = GradientBoostingClassifier(random_state=42)

# Поиск лучших параметров
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Обучение GridSearchCV
grid_search.fit(x_train_split, y_train_split)

# Лучшие параметры
print("Лучшие параметры:", grid_search.best_params_)

# Лучшая модель
best_model = grid_search.best_estimator_

# Оценка на валидационной выборке
val_predictions = best_model.predict(x_val_split)
val_accuracy = accuracy_score(y_val_split, val_predictions)
print(f"Точность на валидационной выборке: {val_accuracy:.4f}")

# Обучение на всех данных
best_model.fit(x_train, y_train)

# Предсказание на тестовых данных
predictions = best_model.predict(x_valid)

# Сохранение результатов
df3 = pd.DataFrame(data={'PassengerId': df2['PassengerId'].tolist(), 'Survived': predictions})
df3.to_csv('my_preds_best_model.csv', index=False)
df3

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Лучшие параметры: {'learning_rate': 0.05, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Точность на валидационной выборке: 0.8212


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
